In [92]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
import pandas as pd

In [93]:
sc = SparkSession.builder.master("local[*]").appName("IK").getOrCreate()

In [94]:
df3 = pd.read_csv("cardio_alco.csv")

In [95]:
df3[["id", "alco"]] = df3["id;alco"].str.split(";", expand = True)

In [96]:
df4 = df3.drop("id;alco", axis = 1)[["id", "alco"]].apply(pd.to_numeric)

In [97]:
df4.dtypes

id      int64
alco    int64
dtype: object

In [79]:
df4.to_csv("alco")

In [80]:
df1 = sc.read.csv("alco", header = True, inferSchema=True)

In [111]:
df2= sc.read.csv("cardio_bases.csv", header= True, inferSchema= True)

In [112]:
df2.describe()

DataFrame[summary: string, id: string, gender: string, height: string, weight: string, ap_hi: string, ap_lo: string, cholesterol: string, smoke: string]

In [113]:
df2 = df2.withColumn("date of birth", to_date(df2.age))

In [115]:
df2.drop("age").show()

+---+------+------+------+-----+-----+-----------+-----+-------------+
| id|gender|height|weight|ap_hi|ap_lo|cholesterol|smoke|date of birth|
+---+------+------+------+-----+-----+-----------+-----+-------------+
|  0|     2|   168|  62.0|  110|   80|          1|    0|   1950-05-10|
|  1|     1|   156|  85.0|  140|   90|          3|    0|   1955-05-19|
|  2|     1|   165|  64.0|  130|   70|          3|    0|   1951-08-17|
|  3|     2|   169|  82.0|  150|  100|          1|    0|   1948-03-31|
|  4|     1|   156|  56.0|  100|   60|          1|    0|   1947-11-03|
|  8|     1|   151|  67.0|  120|   80|          2|    0|   1959-12-30|
|  9|     1|   157|  93.0|  130|   80|          3|    0|   1960-07-16|
| 12|     2|   178|  95.0|  130|   90|          3|    0|   1961-10-30|
| 13|     1|   158|  71.0|  110|   70|          1|    0|   1948-05-15|
| 14|     1|   164|  68.0|  110|   60|          1|    0|   1954-04-20|
| 15|     1|   169|  80.0|  120|   80|          1|    0|   1961-09-06|
| 16| 

In [117]:
df2 = df2.select("id", "date of birth", "height", "weight", "ap_hi", "ap_lo", "cholesterol", "smoke")

In [118]:
df6= df2.join(df1, df1.id == df2.id, how="left")

In [119]:
df6.show()

+---+-------------+------+------+-----+-----+-----------+-----+----+----+----+
| id|date of birth|height|weight|ap_hi|ap_lo|cholesterol|smoke| _c0|  id|alco|
+---+-------------+------+------+-----+-----+-----------+-----+----+----+----+
|  0|   1950-05-10|   168|  62.0|  110|   80|          1|    0|null|null|null|
|  1|   1955-05-19|   156|  85.0|  140|   90|          3|    0|null|null|null|
|  2|   1951-08-17|   165|  64.0|  130|   70|          3|    0|null|null|null|
|  3|   1948-03-31|   169|  82.0|  150|  100|          1|    0|null|null|null|
|  4|   1947-11-03|   156|  56.0|  100|   60|          1|    0|null|null|null|
|  8|   1959-12-30|   151|  67.0|  120|   80|          2|    0|null|null|null|
|  9|   1960-07-16|   157|  93.0|  130|   80|          3|    0|null|null|null|
| 12|   1961-10-30|   178|  95.0|  130|   90|          3|    0|null|null|null|
| 13|   1948-05-15|   158|  71.0|  110|   70|          1|    0|null|null|null|
| 14|   1954-04-20|   164|  68.0|  110|   60|       